# Caches

## Assignment 2 Miscellany

### Due to pace-ice issues, extended to tomorrow @ 9:30.  Assignment 3 will be issued then, too.

### See the clarification announcement on Canvas

### One Further Note:

- Do not be upset if the bandwidth that you get for the CPUs is very different from the report from Intel.
  Unforunately, one model name does not correspond to one memory configuration.

## Quick review: shared memory programming & shared memory hardware

### In shared memory programming we are often writing our code according to a PRAM (Parallel Random Access Memory) Model.  Usually with concurrent reads, and with concurrent write policies being handled in a variety of ways:

- At the algorithm level: trying to write a program where thread writes are guaranteed to be non-concurrent (partitioning the address space, coordination between threads, etc.)

- At the software/runtime level: through mutexes, locks, semaphores, and so on.

- At the hardware level: through atomic operations & transactional memory.

### The strategies available have to take into account that *different threads have different clocks*: we cannot infer which instruction another thread is currently processing.  Strategies for concurrent writes and other synchronization between threads (such as barriers and reductions) in our algorithms must therefore avoid deadlocks.

### The three critical things to understand about code *correctness* when using OpenMP for shared memory programming are:

1. What is the scope of a variable: shared or private?

2. What are the explicit and implicit synchronization semantics of directives (`for`, `single`, `master`, `critical`, `atomic`)?

3. What program state (such as random number generator seeds) needs to persist between `parallel` regions?

### When trying to predict the *performance* of OpenMP code, we must try to understand:

1. What resources a thread is using:
  - Which core/socket? If we don't specify *affinity* (`OMP_PROC_BIND`) it may change over the course of a program

  - Which physical memory? Recall "first touch" placement policy, and *Non Uniform Memory Access*:
    - If we have a streaming kernel, we will be limited by the *minimal bandwidth* in any step between core and
      physical memory. If you are not seeing the bandwidth you expect, maybe the physical memory isn't where you
      think it is.

  - Cache (start discussing today)

2. **When are threads in contention for resources?**:
  - We've already discussed multithreading at the core: if a functional unit does not have the throughput for two
    or more threads' operations per cycle, it will be a bottleneck.
  
  - Cache (start discussing today)

## The same programming & hardware principles apply to shared memory programming on the GPU!

Just a little more so, in some ways.

### Scope Issues:

1. Instead of only `shared` and `private` variables, CUDA has [more](https://wikis.ece.iastate.edu/cpre584/images/3/34/CUDA_Memory.pdf).  Essentially:
  1. `__global__`, `__constant__` visible to all threads
  2. `__shared__` visible to threads in a thread block
  3. (unqualified) visible to just one thread
  
  Example coordination in computing a [reduction](https://devblogs.nvidia.com/faster-parallel-reductions-kepler/)

### Thread synchronization / concurrent write issues:

1. The threads within a warp are always on the same clock (as long as they don't branch diverge!)
2. The threads in a threadblock can be synchronized, such as with the barrier `__syncthreads()`
3. There are atomic operations for some global variable types, like `atomicAdd()`.
4. But the scheduling order / timing of thread blocks to SMs is not directly under control,
   so trying to coordinate between thread blocks without causing deadlocks is tricky.
   
### Which resources is a thread using?  When are they competing

1. The streaming multiprocessor of a thread is fixed for its lifetime: no affinity required.
2. Threads compete for space in the *register file*: register pressure leads to spilling registers to shared
   memory, slower performance.
3. Which physical memory?  We see in assignment 2 how big an effect this can have.
   - Recall that we achieve peak bandwidth on the CPUs only with the benefit of *hardware prefetching* (which helps
     us avoid bubbles in the "load-from-memory" pipeline)
   - The same type of pipelined / asynchronous memory transfers are possible with the GPU, `cudaMemcpyAsync()`:
     pipeline the memory transfer for one kernel while the other is compuing
   - The user also can have direct control over how much shared local memory each thread block uses:
     [see here](https://devblogs.nvidia.com/using-shared-memory-cuda-cc/).
   - (*Folk story ahead*): this kind of explicit memory management was necessary when programming GPUs in the early      days. Thus, people had to think about memory movement, and tried to optimize for it.  They would then compare to CPU programs (which relied on automatic memory management via caches and prefetching,  etc.), where no thought was put into memory management.  100x, 1000x speed ups were reported!  Then people got serious and started comparing optimized CPU code to optimized GPU code.  The results, shockingly, looked much more similar to the hardware characteristics of devices (in terms of peak bandwidth and machine balance).
   - Now, CUDA has implicit memory movement: "Unified Memory" is not only a unified address space for CPUs and GPUs, but it now anticipates your memory movement and duplicates host/device memory on device/host memory as needed.
   - It looks like they've made ... a cache!

## Basic Cache Theory through the Matrix-Matrix Multiplication Example

- Our traditional serial analysis rates an algorithm by $F_f(N)$: the number of operations (flops) it takes to run for a problem of size $N$, ignoring memory movement

- Because of the growing imbalance between flop/s and memop/s per second, it is not unreasonable to ignore the time it takes to do a computation and instead focus on optimizing the number of *loads* and *stores* given that we have a cache--a workspace from which we can compute--of fixed size $Z$.  Let's call this number $W_f(N,Z)$.

- Just as with traditional analysis, we can develop asymptotic lower bounds, and then try to create algorithms that come close to achieving those bounds.

`TODO: whiteboard Matrix Matrix multiplication example`

## Basic Cache Hardware Implementations with Practical Consequences

- [Hager & Wellein](https://moodle.rrze.uni-erlangen.de/pluginfile.php/12220/mod_resource/content/10/01_Arch.pdf), slides 66-96